In [ ]:
import pandas as pd
import numpy as np
import jupyter_black
from typing import Literal, Tuple

import nfl_data_py as nfl
from gspread.exceptions import WorksheetNotFound, SpreadsheetNotFound

from src.modelling.metrics import success_rate_lambda
from src.modelling.utils import guess_the_lines_ovr, get_week_spreads
from src.modelling.homers import (
    google_sheet_names,
    pickers,
    master_google_sheet,
    orchestrate_power_ratings_to_picks,
)
from src.scraping.google_sheets import col_to_int
from src.utils.connections import load_service_account
from src.utils.teams import standardize_teams
from src.visualisation.plots import plot_team_scatter

jupyter_black.load()
pd.set_option("display.max_columns", None)

In [ ]:
week = 2
picker = "Griffin"
# picker = "Harry"
# picker = "Chuck"

In [ ]:
gtl = orchestrate_power_ratings_to_picks(week=week, picker=picker, overwrite_tab=False)

In [ ]:
gtl

In [ ]:
try:
    # open the google sheet
    gsheet = sa.open(sheet_name)
except SpreadsheetNotFound as e:
    print(
        f"WARNING: google sheet '{sheet_name}' not found, ensure it is shared with the service account"
    )
    raise e
# open power ratings for this week
try:
    work_sheet = gsheet.worksheet(tab_name)
    # gsheet.worksheet(f"Wk {week} fail")
except WorksheetNotFound as e:
    print(f"WARNING: tab '{tab_name}' not found")
    raise e

In [ ]:
power_df = pd.DataFrame(work_sheet.get(range_name="A1:P33"))
# reset column names
power_df.columns = power_df.iloc[0]
power_df = power_df.drop(0)
power_df["team"] = power_df["team"].apply(standardize_teams)
power_df = power_df.set_index("team").astype("float32")
assert len(power_df) == 32
gtl = (
    guess_the_lines_ovr(power_df, week=1, season=2024)
    .drop(columns=["away_gpf", "home_gpf"])
    .reset_index(level=0)
)
gtl["adj_line"] = gtl["pred_line"]
gtl["adj_difference"] = None
gtl["adj_pick"] = None
gtl["adj_rank"] = None
col_char_mappings = {col: chr(65 + i) for i, col in enumerate(gtl.columns)}

adj_difference_formulas = []
adj_pick_formulas = []
adj_rank_formulas = []
n = len(gtl)
for index, row in gtl.iterrows():
    i = int(index) + 2
    adj_difference_formulas.append(
        f"={col_char_mappings['adj_line']}{i}-{col_char_mappings['spread_line']}{i}"
    )
    adj_pick_formulas.append(
        f"=IF({col_char_mappings['adj_difference']}{i}>0,{col_char_mappings['home_team']}{i}, {col_char_mappings['away_team']}{i})"
    )
    adj_rank_formulas.append(
        f"=RANK(ABS({col_char_mappings['adj_difference']}{i}), ARRAYFORMULA(ABS(${col_char_mappings['adj_difference']}$2:${col_char_mappings['adj_difference']}${n+1})))"
    )
gtl["adj_difference"] = adj_difference_formulas
gtl["adj_pick"] = adj_pick_formulas
gtl["adj_rank"] = adj_rank_formulas

In [ ]:
pick_tab = tab_name + " - Picks"
pick_tab_exists = True
try:
    pick_worksheet = gsheet.worksheet(pick_tab)
except WorksheetNotFound:
    pick_tab_exists = False

if not pick_tab_exists or overwrite_tab:
    # gsheet.del_worksheet(gsheet.worksheet(f"Wk {week} - Picks"))
    pick_worksheet = gsheet.add_worksheet(pick_tab, 100, 100)
    # Convert DataFrame to list of lists
    data_to_upload = gtl.map(
        lambda x: round(x, 2) if isinstance(x, float) else x
    ).values.tolist()
    # Optionally, add header if needed
    data_to_upload.insert(0, gtl.columns.tolist())
    # Update the worksheet with formulas
    # If update is causing issues, ensure that the values are treated correctly
    # by setting values explicitly as formulas
    cell_range = (
        f"A1:{chr(65 + len(gtl.columns) - 1)}{len(gtl) + 1}"  # Adjust range as needed
    )
    pick_worksheet.update(values=data_to_upload, range_name=cell_range, raw=False)

    if hide_cols:
        i = col_to_int(col_char_mappings["spread_line"])
        pick_worksheet.hide_columns(i, i + 1)

        i = col_to_int(col_char_mappings["difference"])
        j = col_to_int(col_char_mappings["rank"])
        pick_worksheet.hide_columns(i, j + 1)
        i = col_to_int(col_char_mappings["adj_difference"])
        j = col_to_int(col_char_mappings["adj_rank"])
        pick_worksheet.hide_columns(i, j + 1)

else:
    print(f"Picks tab {pick_tab} already exists and overwrite_tab set to False")